In [1]:
import os
print os.environ["DISPLAY"]
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams['figure.figsize'] = (8.0, 8.0)
import pandas as pd
import pickle
import scipy.io as sio
from shdom.generate_atmosphere import TemperatureProfile
from shdom.inverse import *
from shdom.sensors import SensorNetwork
from shdom.utils import loadRGBmeasurements, setupSimulationParameters

KeyError: 'DISPLAY'

## Load the Model 

Note, there is a need to create a model object before loading it.

In [ ]:
DATA_PATH = "2017_04_22_08_30_00"
SOLAR_FLUX = 1.6658941228097737
SHDOM_COEFF = 0.01

base_path = os.path.join('../saved_states/camera_network', DATA_PATH)
config_path = '../configs/atmospheres/camera_network/phase_from_aeronet.yaml'

source_params, temp_profile = setupSimulationParameters(base_path, SOLAR_FLUX)
model = InverseModel(
    config_path=config_path,
    source_params=source_params
)
model.setGrid(base_path, delxy=0.25, num_cam_thresh=3)
camera_network = SensorNetwork(data_dir=base_path)

In [ ]:
#model.load_inversion_state('../tests/results/130+/1705071425_4e5c/inversion_state.pkl')
#model.load_inversion_state('../tests/results/130+/1705080309_fac9/inversion_state.pkl')
#odel.load_inversion_state('../tests/results/131+/1705081122_53a4/inversion_state.pkl')
#model.load_inversion_state('../tests/results/131+/1705081339_aa7f/inversion_state.pkl')
#model.load_inversion_state('../tests/results/132+/1705082345_976a/inversion_state.pkl')
#model.load_inversion_state('../tests/results/133+/1705090719_ef4f/inversion_state_tmp.pkl')
#model.load_inversion_state('../tests/results/133+/1705090720_2cd1/inversion_state_tmp.pkl')
#model.load_inversion_state('../scripts/results/134+/1705111513_9ada/inversion_state_tmp.pkl')
#model.load_inversion_state('../scripts/results/134+/1705111503_432e/inversion_state_tmp.pkl')
#model.load_inversion_state('../scripts/results/134+/1705141437_0636/inversion_state_tmp.pkl')
#model.load_inversion_state('../scripts/results/134+/1705141546_ff2a/inversion_state_tmp.pkl')
#model.load_inversion_state('../scripts/results/134+/1705141610_d1d9/inversion_state_tmp.pkl')
#model.load_inversion_state('../scripts/results/135/1705142119_7a66/inversion_state_tmp.pkl')
#model.load_inversion_state('../scripts/results/137+/1705291622_db54/inversion_state_tmp.pkl')
#model.load_inversion_state('../scripts/results/137+/1705310137_cc46/inversion_state_tmp.pkl')
#model.load_inversion_state('../scripts/results/137+/1705310906_770d/inversion_state_tmp.pkl')
#model.load_inversion_state('../scripts/results/137+/1705311128_636a/inversion_state_tmp.pkl')
#model.load_inversion_state("../scripts/results/137+/1706010051_34b6/inversion_state_tmp.pkl")
#model.load_inversion_state("../scripts/results/137+/1706011045_39e6/inversion_state_tmp.pkl")
#model.load_inversion_state("../scripts/results/137+/1706011046_3f54/inversion_state_tmp.pkl")
#model.load_inversion_state("../scripts/results/138/1706050633_c956/inversion_state_tmp.pkl")
#model.load_inversion_state("../scripts/results/138+/1706080722_512c/inversion_state_tmp.pkl")
model.load_inversion_state("../scripts/results/138+/1706082324_f17f/inversion_state_tmp.pkl")

### There is a need to create the internal structures of SHDOM

In [ ]:
model.initMieTables(refractive_index_flag=True)
model.computeOpticalFields()
model.solve()

### Load the original measurements

In [ ]:
measurements, masks = loadRGBmeasurements(
    base_path, shdom_coeff=SHDOM_COEFF)

## Plot the Reconstructed vs. Original Images

In [ ]:
imgs = camera_network.getImages(model)

In [ ]:
#
# First calculate a value scale for all images.
#
vmax = 0
for img, measurement, mask in zip(imgs, measurements, masks):
    imax = img.max()
    mmax = (measurement[0] * mask).max()
    vmax = max(vmax, max(imax, mmax))
        
for img, measurement, mask, camera_id in zip(imgs, measurements, masks, camera_network.camera_ids):
    f, axes = plt.subplots(ncols=2, figsize=(16, 8))
    axes[0].imshow(img, vmax=vmax/10, origin="bottom")
    axes[1].imshow(measurement[0] * mask, vmax=vmax/10, origin="bottom")
    f.suptitle(camera_id)

### 3D visualization

In [ ]:
import mayavi.mlab as mlab

def viz3D(data, model, cameras_data):
    mlab.figure()
    X, Y, Z = np.meshgrid(model.shdom._xgrid[:-1], model.shdom._ygrid[:-1], model.shdom._zgrid, indexing='ij')
    src = mlab.pipeline.scalar_field(X, Y, Z, data)
    #src.spacing = [1, 1, 1]
    src.update_image_data = True
    ipw_x = mlab.pipeline.image_plane_widget(src, plane_orientation='x_axes')
    ipw_y = mlab.pipeline.image_plane_widget(src, plane_orientation='y_axes')
    ipw_z = mlab.pipeline.image_plane_widget(src, plane_orientation='z_axes')
    colorbar = mlab.colorbar(title=r'${\rm km}^{-1}$', orientation='horizontal', label_fmt = '%.2f')
    mlab.outline()

    for camera_id, (x, y, z) in cameras_data.items():
        #
        # Draw a point at the camera center.
        #
        mlab.points3d(
            [x], [y], [z],
            color=(1, 0, 0), mode='sphere', scale_mode='scalar', scale_factor=.1,
        )

        mlab.text3d(x, y, z+0.2, camera_id, color=(0, 0, 0), scale=.1)

    mlab.show()

In [ ]:
cameras_data = {}
for x, y, z, camera_id in zip(
    camera_network.x_array,
    camera_network.y_array,
    camera_network.z_array,
    camera_network.camera_ids,
    ):
    cameras_data[camera_id] = x.flatten()[0], y.flatten()[0], z.flatten()[0]

In [ ]:
with open("2017_04_22_09_10_00_grid_0.25.pkl", "wb") as f:
    pickle.dump((model.beta_c, cameras_data, model.shdom._xgrid[:-1], model.shdom._ygrid[:-1], model.shdom._zgrid), f)

In [ ]:
viz3D(model.beta_c, model, cameras_data)

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
@interact(depth=(0, model.beta_c.shape[-1]-1))
def z_slice(depth):
    plt.figure()
    plt.imshow(model.beta_c[..., depth], origin="bottom")
    plt.show()

## Average the grid along X-Y plane

In [ ]:
plt.plot(model.shdom._zgrid, model.beta_c.mean(axis=0).mean(axis=0))
plt.xlabel("Height [km]")
plt.ylabel("Extinction [1/km]")
plt.xlim(0, 10)
plt.ylim(0, 0.14)

In [ ]:
def gaussian(center_x, center_y, height=1., width_x=0.1, width_y=0.1):
    """Returns a gaussian function with the given parameters"""

    def foo(x, y):
        g = height*np.exp(-(((center_x-x)/width_x)**2+((center_y-y)/width_y)**2)/2)
        return g/g.sum()
    
    return foo


X, Y = np.meshgrid(np.arange(-25, 24), np.arange(-25, 24))
g = gaussian(0, 0, width_x=4, width_y=4)(X, Y)
plt.imshow(g)
print g.sum()

In [ ]:
profile = []
for i in range(model.beta_c.shape[-1]):
    profile.append((g * model.beta_c[:,:,i]).sum())
    
plt.plot(model.shdom._zgrid, profile)
plt.xlabel("Height [km]")
plt.ylabel("Extinction [1/km]")
plt.xlim(0, 10)

## Cost

In [ ]:
plt.plot(model.cost)
plt.xlabel("Iteration")
plt.ylabel("Cost")

## Ground truth LIDAR data

In [ ]:
lidar_df = pd.read_csv(r"../../cameranetwork/data/lidar/2017_04_22/-20170422_0800-0900-21smooth.txt", delimiter="\t", index_col=0)
lidar_df.head()

In [ ]:
plt.figure()
lidar_df[lidar_df["alpha532 (Mm^-1 sr^-1):"]<0] = 0
plt.plot(lidar_df["alpha532 (Mm^-1 sr^-1):"].index, lidar_df["alpha532 (Mm^-1 sr^-1):"]/1000, label="LIDAR data")
plt.plot(model.shdom._zgrid, profile, label="Narrow mean")
plt.plot(model.shdom._zgrid, model.beta_c.mean(axis=0).mean(axis=0), label="Full mean")
plt.ylabel("Extinction [1/km]", fontsize=20)
plt.xlabel("Height [km]", fontsize=20)
plt.xlim(0, 6)
plt.legend()